In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import *
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    
    
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 35ms/step - loss: 2.5975 - accuracy: 0.6681 - precision: 0.6559 - recall: 0.6145 - auc: 0.6988
Epoch 2/10
29/29 [==============================] - 1s 26ms/step - loss: 0.2277 - accuracy: 0.9255 - precision: 0.9110 - recall: 0.9322 - auc: 0.9727
Epoch 3/10
29/29 [==============================] - 1s 27ms/step - loss: 0.0645 - accuracy: 0.9814 - precision: 0.9724 - recall: 0.9883 - auc: 0.9991
Epoch 4/10
29/29 [==============================] - 1s 20ms/step - loss: 0.0225 - accuracy: 0.9945 - precision: 0.9930 - recall: 0.9953 - auc: 1.0000
Epoch 5/10
29/29 [==============================] - 1s 31ms/step - loss: 0.0112 - accuracy: 0.9978 - precision: 1.0000 - recall: 0.9953 - auc: 1.0000
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 0.0084 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000
Epoch 7/10
29/29 [==============================] - 1s 22ms/step - loss: 0.0035 - accuracy: 1.0000 -

29/29 [==============================] - 1s 27ms/step - loss: 0.0098 - accuracy: 0.9978 - precision_4: 0.9977 - recall_4: 0.9977 - auc_4: 1.0000
Epoch 10/10
29/29 [==============================] - 1s 22ms/step - loss: 0.0120 - accuracy: 0.9978 - precision_4: 0.9977 - recall_4: 0.9977 - auc_4: 1.0000
 Accuracy: 93.275% (+/-1.313) 
 Precision: 90.153% (+/-4.607) 
 Recall: 93.182% (+/-5.183) 
 AUC: 97.806% (+/-0.353) 


In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 35ms/step - loss: 9.6118 - accuracy: 0.4786 - precision_5: 0.4294 - recall_5: 0.3411 - auc_5: 0.4493
Epoch 2/10
29/29 [==============================] - 1s 30ms/step - loss: 0.6930 - accuracy: 0.5312 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - auc_5: 0.5000
Epoch 3/10
29/29 [==============================] - 1s 30ms/step - loss: 0.6929 - accuracy: 0.5312 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - auc_5: 0.5000
Epoch 4/10
29/29 [==============================] - 1s 27ms/step - loss: 0.6927 - accuracy: 0.5312 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - auc_5: 0.5000
Epoch 5/10
29/29 [==============================] - 1s 33ms/step - loss: 0.6926 - accuracy: 0.5312 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - auc_5: 0.5000
Epoch 6/10
29/29 [==============================] - 1s 27ms/step - loss: 0.6925 - accuracy: 0.5312 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - auc_5: 0.5000
Epoch 7/10
29/29 [======

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 32ms/step - loss: 2.5022 - accuracy: 0.5575 - precision_10: 0.5392 - recall_10: 0.3855 - auc_10: 0.5746
Epoch 2/10
29/29 [==============================] - 1s 25ms/step - loss: 0.3648 - accuracy: 0.8456 - precision_10: 0.8509 - recall_10: 0.8131 - auc_10: 0.9244
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1543 - accuracy: 0.9518 - precision_10: 0.9528 - recall_10: 0.9439 - auc_10: 0.9854
Epoch 4/10
29/29 [==============================] - 1s 22ms/step - loss: 0.0808 - accuracy: 0.9781 - precision_10: 0.9789 - recall_10: 0.9743 - auc_10: 0.9941
Epoch 5/10
29/29 [==============================] - 1s 28ms/step - loss: 0.0292 - accuracy: 0.9956 - precision_10: 0.9907 - recall_10: 1.0000 - auc_10: 0.9983
Epoch 6/10
29/29 [==============================] - 1s 27ms/step - loss: 0.0111 - accuracy: 0.9989 - precision_10: 0.9977 - recall_10: 1.0000 - auc_10: 1.0000
Epoch 7/10
29/29 [============================